In [11]:
import pandas as pd

### Import Data

In [76]:
fileName = 'loan_data.csv'
df = pd.read_csv(fileName)
df.head()

C:\Users\SenithDilitha\AppData\Local\Temp\ipykernel_16624\1959362264.py:2: DtypeWarning: Columns (18,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fileName)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,112435993,NaN,2300,2300,2300.0,36 months,12.62,77.08,C,C1,...,0.0,NaN,1.0,2.0,2315.0,55.0,7100.0,1.0,0.0,2.0
1,112290210,NaN,16000,16000,16000.0,60 months,12.62,360.95,C,C1,...,11078.0,69.0,3.0,5.0,1962.0,94.0,31900.0,0.0,6.0,1.0
2,112436985,NaN,6025,6025,6025.0,36 months,15.05,209.01,C,C4,...,0.0,NaN,1.0,2.0,1950.0,45.0,27700.0,1.0,5.0,3.0
3,112439006,NaN,20400,20400,20400.0,36 months,9.44,652.91,B,B1,...,53566.0,95.0,1.0,2.0,4240.0,60.0,46900.0,1.0,1.0,1.0
4,112438929,NaN,13000,13000,13000.0,36 months,11.99,431.73,B,B5,...,8466.0,72.0,0.0,1.0,2996.0,78.0,7800.0,0.0,0.0,0.0


### Clean Data


In [70]:
df = df.drop_duplicates()


df_modified = df.drop(columns=['id', 
                               'member_id', 'desc', 'out_prncp_inv',                                  #empty
                               'funded_amnt', 'funded_amnt_inv', 'sub_grade', 'purpose',              #repeated columns
                               'out_prncp_inv', 'total_pymnt_inv',            
                               'emp_title', 'title', 'home_ownership', 'zip_code', 'addr_state',      #Categorical features (to be included later)
                               'pymnt_plan', 'issue_d', 'initial_list_status', 'total_rec_late_fee',  #Same value columns
                               'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 
                               'last_credit_pull_d', 'collections_12_mths_ex_med',                
                               'earliest_cr_line', 'verification_status', 'out_prncp',                #Not relevant
                               'mths_since_last_record', 'pub_rec',
                               'next_pymnt_d', 'policy_code', 'application_type', 'annual_inc_joint',
                               'dti_joint', 'verification_status_joint'])

df_modified['term'] = pd.to_numeric(df_modified['term'].str.replace(' months', ''))                              # 36 months -----> 36

df_modified['emp_length'] = df_modified['emp_length'].str.extract(r'(\d+)')                                      # 10+ years -----> 10, 7 years ---> 7
df_modified['emp_length'] = pd.to_numeric(df_modified['emp_length'])

df_modified['loan_status'] = df_modified['loan_status'].str.replace(pat='Late.*', regex=True, repl='Late')        # Late (15-30 days) ---> Late
df_modified['loan_status'] = df_modified['loan_status'].str.replace(pat='Fully Paid', regex=True, repl='Current') # Fully Paid ---> Current

df_modified = df_modified[df_modified['loan_status'] != 'In Grace Period']                                        # Remove loans in Grace Period

mapping = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6}                                                        # Map grade to a number
df_modified['grade'] = df_modified['grade'].map(mapping)

df_modified

,loan_amnt,term,int_rate,installment,grade,emp_length,annual_inc,loan_status,dti,delinq_2yrs,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,2300,36,12.62,77.08,3.0,NaN,10000.0,Current,21.61,0,...,0.0,NaN,1.0,2.0,2315.0,55.0,7100.0,1.0,0.0,2.0
1,16000,60,12.62,360.95,3.0,10.0,94000.0,Current,25.61,0,...,11078.0,69.0,3.0,5.0,1962.0,94.0,31900.0,0.0,6.0,1.0
2,6025,36,15.05,209.01,3.0,7.0,46350.0,Current,8.88,0,...,0.0,NaN,1.0,2.0,1950.0,45.0,27700.0,1.0,5.0,3.0
3,20400,36,9.44,652.91,2.0,10.0,44000.0,Current,27.06,0,...,53566.0,95.0,1.0,2.0,4240.0,60.0,46900.0,1.0,1.0,1.0
4,13000,36,11.99,431.73,2.0,10.0,85000.0,Current,6.79,1,...,8466.0,72.0,0.0,1.0,2996.0,78.0,7800.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759333,6000,36,7.89,187.72,1.0,1.0,38000.0,Current,12.35,0,...,NaN,NaN,NaN,NaN,NaN,NaN,21400.0,NaN,NaN,NaN
759334,6000,36,9.17,191.28,2.0,NaN,32640.0,Current,22.76,0,...,NaN,NaN,NaN,NaN,NaN,NaN,18600.0,NaN,NaN,NaN
759335,14400,60,13.18,328.98,3.0,10.0,47000.0,Late,19.64,0,...,NaN,NaN,NaN,NaN,NaN,NaN,18300.0,NaN,NaN,NaN
759336,34050,36,15.41,1187.21,4.0,10.0,87800.0,Current,12.10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,47900.0,NaN,NaN,NaN


In [77]:
df['loan_status'].value_counts()

loan_status
Current               565523
Fully Paid            130718
Charged Off            37197
Late (31-120 days)     15354
In Grace Period         6634
Late (16-30 days)       3876
Default                   36
Name: count, dtype: int64

In [79]:
df_modified.describe()

,loan_amnt,term,int_rate,installment,grade,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,752704.000000,752704.000000,752704.000000,752704.000000,747284.000000,702701.000000,7.527040e+05,752351.000000,752704.000000,752703.000000,...,7.526440e+05,652838.000000,752644.000000,752644.000000,752644.000000,752592.000000,7.527040e+05,752644.000000,752643.000000,752643.000000
mean,14692.241173,42.250872,13.166569,441.994358,2.655499,6.051716,7.997583e+04,18.971717,0.356549,0.537581,...,3.569869e+04,69.996578,1.339107,2.846491,5793.914519,59.214468,3.452881e+04,0.967063,1.473417,2.104744
std,9211.745005,10.533170,5.046621,275.463599,1.170226,3.610643,1.640532e+05,13.320278,0.949800,0.837643,...,4.322195e+04,23.660404,1.520899,2.620603,5585.012393,20.475522,3.518844e+04,1.481543,2.678222,2.399596
min,1000.000000,36.000000,5.320000,30.120000,1.000000,1.000000,0.000000e+00,-1.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7500.000000,36.000000,9.750000,241.100000,2.000000,2.000000,4.800000e+04,12.250000,0.000000,0.000000,...,9.298000e+03,56.000000,0.000000,1.000000,2371.000000,46.000000,1.460000e+04,0.000000,0.000000,0.000000
50%,12000.000000,36.000000,12.620000,370.480000,3.000000,6.000000,6.700000e+04,18.170000,0.000000,0.000000,...,2.359500e+04,73.000000,1.000000,2.000000,4420.000000,60.000000,2.530000e+04,0.000000,0.000000,1.000000
75%,20000.000000,60.000000,15.590000,589.220000,3.000000,10.000000,9.500000e+04,24.770000,0.000000,1.000000,...,4.631900e+04,86.000000,2.000000,4.000000,7503.000000,74.000000,4.300000e+04,1.000000,2.000000,3.000000
max,40000.000000,60.000000,30.990000,1719.830000,6.000000,10.000000,1.100000e+08,999.000000,42.000000,5.000000,...,1.711009e+06,1000.000000,28.000000,60.000000,776843.000000,211.000000,2.175000e+06,48.000000,111.000000,49.000000


### Feature Selection using statistics


In [72]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif #ANOVA Correlation Coefficient 
from sklearn.impute import SimpleImputer

# Numerical Feature Selection
X = df_modified[['loan_amnt', 'term', 'int_rate', 'installment', 'annual_inc', 'grade', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
                 'total_bal_il', 'total_rev_hi_lim', 'dti', 'revol_bal', 'revol_util', 'tot_cur_bal', 'last_pymnt_amnt']]
y = df_modified['loan_status']

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

fs = SelectKBest(score_func=f_classif, k=4)

X_selected = fs.fit_transform(X_imputed, y)

selected_features_indices = fs.get_support(indices=True)

# Get the names of the selected features
selected_features_names = X.columns[selected_features_indices]
print("Selected Numerical features:", selected_features_names)


# Print scores for each feature
sorted_features = sorted(zip(X.columns, fs.scores_), key=lambda x: x[1], reverse=True)
print("Scores for each feature:")
for feature, score in sorted_features:
    print(f"{feature}: {score}")



Selected Numerical features: Index(['int_rate', 'grade', 'open_rv_24m', 'last_pymnt_amnt'], dtype='object')
Scores for each feature:
grade: 8335.450619307772
int_rate: 7760.589112413699
last_pymnt_amnt: 2274.525488179858
open_rv_24m: 1158.5172974888103
open_rv_12m: 758.1101902574812
installment: 601.7952338666246
dti: 368.17793069030466
total_rev_hi_lim: 366.49200684676856
term: 353.42266772618757
tot_cur_bal: 348.74279518586025
revol_util: 268.73575195865277
loan_amnt: 235.15374378528912
max_bal_bc: 202.22587149673586
revol_bal: 55.660157143515676
annual_inc: 27.5251839948866
total_bal_il: 25.811978250068748


### Create Model and Test

In [73]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['loan_status'] = label_encoder.fit_transform(df['loan_status'])

X_train, X_test, y_train, y_test = train_test_split(X_selected, y,)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

predictions = model.predict(X_test)
score = accuracy_score(y_test, predictions)
print(score)

0.8995461695434062
